In [2]:
%load_ext autoreload
%autoreload 2

from model import model, loss_function
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load dataset

In [3]:
from dataLoader import DataLoader
top_k = 3000
data_path = '/data/Niklas/Flickr30k'
dataLoader = DataLoader(data_path)
dataLoader.convert_to_dataset(top_k)
if len(dataLoader.tokenizer.index_word) < top_k:
    top_k = len(dataLoader.tokenizer.index_word) - 1 # If our dictonary is less than top_k. -1 because <unk> is included
    dataLoader.top_k = top_k
max_length = dataLoader.max_length

In [5]:
print("Training size: {0}".format(len(dataLoader.cap_train)))
print("Validation size: {0}".format(len(dataLoader.cap_val)))
print("Max length: {0}".format(max_length))
print("Top k: {0}".format(top_k))

Training size: 116000
Validation size: 29000
Max length: 80
Top k: 3000


# Create & train model

In [6]:
from create_dataset import create_dataset
import tensorflow as tf

image_shape = (299,299,3)
embedding_matrix = None
embedding_dim = 512
lstm_units = embedding_dim

buffer_size = 1000
batch_size = 16
train_dataset = create_dataset(dataLoader.img_name_train, dataLoader.cap_train, batch_size, buffer_size)
val_dataset = create_dataset(dataLoader.img_name_val, dataLoader.cap_val, batch_size, buffer_size)
num_batches = int(len(dataLoader.cap_train)/batch_size)

print("Batches: {0}, Steps per epochs: {1}".format(batch_size, num_batches))

Batches: 16, Steps per epochs: 7250


In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath='./Saved_models/best_model',
        save_weights_only=True,
        monitor='val_loss',
        mode='min',
        save_best_only=True)

mirrored_strategy = tf.distribute.MirroredStrategy()
batch_size *= mirrored_strategy.num_replicas_in_sync
train_batches = int(len(dataLoader.cap_train)/batch_size)
val_batches = int(len(dataLoader.cap_val)/batch_size)
with mirrored_strategy.scope():
    m = model(image_shape, embedding_dim, lstm_units, top_k, max_length)
    m.compile(optimizer = tf.keras.optimizers.Adam(), loss = loss_function)
    history = m.fit(train_dataset, 
                    epochs = 10,
                    steps_per_epoch = train_batches,
                    shuffle=True,
                    callbacks = [model_checkpoint_callback],
                    validation_data = val_dataset,
                    validation_steps = val_batches)
    m.save_weights('./Models/model')

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [ ]:
m.summary()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(m.history.history['loss'])
plt.plot(m.history.history['val_loss'])
plt.title("Traning")
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.legend(["traning", "validation"])

# Load trained model

In [ ]:
m = model(image_shape, embedding_dim, lstm_units, top_k, max_length)
m.load_weights('./Saved_models/model')

# Predict on test data

In [ ]:
from create_dataset import read_img
import tensorflow as tf
import matplotlib.pyplot as plt

sample = dataLoader.img_name_train[0]
cap = dataLoader.org_data[sample]
img = read_img(sample)

img_1 = tf.io.read_file(sample)
img_1 = tf.image.decode_jpeg(img_1, channels=3)
plt.imshow(img_1)

result = m.predict(img[None,...], dataLoader.tokenizer)
print(' '.join(result))
[print(c) for c in cap]

# Predict on val data

In [ ]:
from create_dataset import read_img
import tensorflow as tf
import matplotlib.pyplot as plt

sample = dataLoader.img_name_val[0]
cap = dataLoader.org_data[sample]
img = read_img(sample)

img_1 = tf.io.read_file(sample)
img_1 = tf.image.decode_jpeg(img_1, channels=3)
plt.imshow(img_1)

result = m.predict(img[None,...], dataLoader.tokenizer)
print(' '.join(result))
[print(c) for c in cap]

# Test on own imag

In [ ]:
from create_dataset import read_img
import tensorflow as tf
import matplotlib.pyplot as plt

sample = ""
cap = dataLoader.org_data[sample]
img = read_img(sample)

img_1 = tf.io.read_file(sample)
img_1 = tf.image.decode_jpeg(img_1, channels=3)
plt.imshow(img_1)

result = m.predict(img[None,...], dataLoader.tokenizer)
print(' '.join(result))
[print(c) for c in cap]